In [1]:
import pandas as pd
import pyodbc 

In [14]:
def connection_db(user, password):
    dbServerName         = 'minhabase.database.windows.net'
    dbName               = 'nomebanco'
    dbUserName           = "username"
    dbPassword           = dbutils.secrets.get (scope= 'scope', key = 'svr-secret')
    driver               = 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
    server_name = f"jdbc:sqlserver://{dbServerName}:1433;database={dbName}"

In [ ]:
def ingest_data_to_sql(dataframe, table_name, connection):

    dataframe.to_sql(table_name, connection, if_exists='append', index=False)

    connection.close()

In [ ]:
def convert_datetime(connection, table_name, column_name):
    query = f"SELECT [{column_name}] FROM {table_name}"
    dataframe = pd.read_sql(query, connection)

    dataframe[column_name] = pd.to_datetime(dataframe[column_name], errors='coerce')

    dataframe.to_sql(table_name, connection, if_exists='replace', index=False)
    
    connection.close()

In [ ]:
def convert_numeric(connection, dataframe, col_name):
    query = f"SELECT [{column_name}] FROM {table_name}"
    dataframe = pd.read_sql(query, connection)

    dataframe[column_name] = pd.to_int(dataframe[column_name], errors='coerce')

    dataframe.to_sql(table_name, connection, if_exists='replace', index=False)
    
    connection.close()

In [ ]:
def replace(dataframe, col_name):
    if col_name not in dataframe.columns:
        raise ValueError(f"A coluna '{col_name}' não existe no DataFrame.")

    replacements = {"1": None, "Mr.": "Sr.", "Ms.": "Sra.", "Ms": "Sra.", "Mrs.": "Sra."}

    dataframe[col_name] = dataframe[col_name].replace(replacements)

    return dataframe

In [ ]:
dbutils.fs.mount(\
    source = "wasbs://camilayzidoro@blobname.blob.core.windows.net",\
    mount_point = "/mnt/camilayzidoro",\
    extra_configs = {"fs.azure.account.key.blobname.blob.core.windows.net":dbutils.secrets.get(scope = 'scope', key = "stg-secret")})


mount_point = "/mnt/camilayzidoro"

df_person = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Person.Person.csv")

In [ ]:
replace(df_person, 'Title')

print(df_person) 

In [ ]:
ingest_data_to_sql(df_person, '[Person].Person')

In [ ]:
mount_point = "/mnt/camilayzidoro"

df_product = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Production.Product.csv")

In [ ]:
convert_datetime(connection_db, df_product, 'SellStartDate')
convert_numeric(connection_db, df_product, 'ProductSubcategoryID')
convert_numeric(connection_db, df_product, 'ProductLine')
convert_numeric(connection_db, df_product,'DaysToManufacture')
convert_numeric(connection_db, df_product, 'Weight')
convert_numeric(connection_db, df_product,'SafetyStockLevel')
replace(df_product,'Color')

print(df_product)

In [ ]:
ingest_data_to_sql(df_product, '[Production].product')

In [ ]:

mount_point = "/mnt/camilayzidoro"

df_salesCustomer = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Sales.Customer.csv")

In [ ]:
ingest_data_to_sql(df_salesCustomer, '[Sales].Customer')

In [ ]:
mount_point = "/mnt/camilayzidoro"

df_specialOfferProduct = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Sales.SpecialOfferProduct.csv")

In [ ]:
ingest_data_to_sql(df_specialOfferProduct, '[Sales].SpecialOfferProduct')

In [ ]:
mount_point = "/mnt/camilayzidoro"

df_salesOrderHeader = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Sales.SalesOrderHeader.csv")

In [ ]:
ingest_data_to_sql(df_salesOrderHeader, '[Sales].SalesOrderHeader')

In [ ]:
mount_point = "/mnt/camilayzidoro"

df_salesOrderDetail = spark.read.format("csv").options(header='true', delimiter = ';').load(f"{mount_point}/Sales.SalesOrderDetail.csv")

In [ ]:
ingest_data_to_sql(df_salesOrderDetail, '[Sales].SalesOrderDetail')

In [ ]:
dbutils.fs.unmount("/mnt/camilayzidoro")